In [1]:
import os
import numpy as np
import random
from scipy import ndimage
from skimage import transform
import keras
from keras.layers import Dense, Lambda
from keras.applications import VGG16
from keras.models import Model
from keras.optimizers import SGD
from keras.callbacks import Callback, ModelCheckpoint, EarlyStopping
import skimage.io as io

# MEAN = np.array([123.0, 117.0, 104.0])
MEAN = np.array([0, 0, 0])
def getModel( output_dim ):
    ''' 
        * output_dim: the number of classes (int)
        
        * return: compiled model (keras.engine.training.Model)
    '''
    vgg_model = VGG16( weights='imagenet', include_top=True )
#     vgg_out = vgg_model.layers[-2].output #Last FC layer's output 
    layer_name = 'fc2'
    x = vgg_model.get_layer(layer_name).output
    temper = 4.0;
    x = Lambda(lambda x: x/temper)(x)
    
    layer_name = 'predictions'
    pred_layer = vgg_model.get_layer(layer_name) 
    weight = pred_layer.get_weights()
    x = Dense(1000, activation='softmax', name='softmax1', weights=weight)(x) 
    
    softmax_layer = Dense(256, activation='softmax')(x) #Create softmax layer taking input as vgg_ou
    #Create new transfer learning model
    tl_model = Model( input=vgg_model.input, output=softmax_layer )

    #Freeze all layers of VGG16 and Compile the model
    #Confirm the model is appropriate
    for l in vgg_model.layers:
        l.trainable = False
        
    layer_name = 'softmax1'
    tl_model.get_layer(layer_name).trainable = False
#     tl_model.get_layer(layer_name).set_weights(weights)
    
    tl_model.summary()

    return tl_model
def convert_image(collect):
#     sz = np.shape(collect)[0]
    sz = len(collect)
    images = np.zeros((sz, 224, 224, 3))
    for i in range(sz):
        img = collect[i]
        img = transform.resize(img, (224, 224))*255
        if (len(img.shape) != 3):
           img = img[:,:,np.newaxis]
        images[i] = img
    return images

sizeOfEachClass = 16
sizeOfTestEachClass = 16
path='/home/yiluo/Homework/253/3/256_ObjectCategories/'
files = os.listdir(path)
X_train, X_val, X_test = [], [], []
Y_train, Y_val, Y_test = [], [], []
label = 0
for category in files:
    sub_path = path + category
    sub_files = os.listdir(sub_path)
    cnt = np.shape(sub_files)[0]
    num = 1
    for img_nm in sub_files:
        if img_nm[-4:] == '.jpg':
            img_path = sub_path + '/' + img_nm
            #img = ndimage.imread(sub_path + '/' + img_nm)
            #img = transform.resize(img, (224, 224))*255
            if num <= np.floor(0.8*cnt):
                if num <= sizeOfEachClass:
                    X_train.append(img_path)
                    Y_train.append(label)
            elif num <= np.floor(0.9*cnt):
                if num <= np.floor(0.8*cnt) + sizeOfTestEachClass:
                    X_val.append(img_path)
                    Y_val.append(label)
            else:
                X_test.append(img_path)
                Y_test.append(label)
            num += 1
    label += 1

Using Theano backend.
Using gpu device 0: GeForce GTX 1080 (CNMeM is disabled, cuDNN not available)


In [3]:
#shuffle
ind=range(int(len(X_train)))
random.shuffle(ind)
X_train = [X_train[i] for i in ind]
Y_train = [Y_train[i] for i in ind]
Y_train_label = np.zeros((len(Y_train), 256))
for i in range(len(Y_train)):
    Y_train_label[i, Y_train[i]] = 1
    
#validation  
Y_val_label = np.zeros((len(Y_val), 256))
for i in range(len(Y_val)):
    Y_val_label[i, Y_val[i]] = 1
    
#test
Y_test_label = np.zeros((len(Y_test), 256))
for i in range(len(Y_test)):
    Y_test_label[i, Y_test[i]] = 1

In [6]:
#Output dim for your dataset
output_dim = 256 #For Caltech256

#Data
coll = io.ImageCollection(X_train)
X_train_data = convert_image(coll)
# coll_val = io.ImageCollection(X_val)
# X_val_data = convert_image(coll_val)

In [41]:
#Model
tl_model = getModel( output_dim )
# rmsprop = keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0) # error

# tl_model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

/usr/local/lib/python2.7/dist-packages/Keras-1.2.2-py2.7.egg/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_7 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 4096)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_7 (InputLayer)             (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
block1_conv1 (Convolution2D)     (None, 224, 224, 64)  1792        input_7[0][0]                    
____________________________________________________________________________________________________
block1_conv2 (Convolution2D)     (None, 224, 224, 64)  36928       block1_conv1[0][0]               
____________________________________________________________________________________________________
block1_pool (MaxPooling2D)       (None, 112, 112, 64)  0           block1_conv2[0][0]               
___________________________________________________________________________________________

In [42]:
# sgd = keras.optimizers.SGD(lr=6, momentum=0.9, decay=0.0, nesterov=True)
# tl_model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

adagrad = keras.optimizers.Adagrad(lr=1, epsilon=1e-08, decay=0.0)
tl_model.compile(loss='categorical_crossentropy', optimizer=adagrad, metrics=['accuracy'])

In [43]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    samplewise_center=True)

# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
datagen.fit(X_train_data)

In [44]:
hist = tl_model.fit(X_train_data, Y_train_label, 
             batch_size=32, nb_epoch=20, 
             verbose=1, validation_split=0.5,
             shuffle=None, class_weight=None, sample_weight=None)

Train on 2048 samples, validate on 2048 samples
Epoch 1/20
2048/2048 [==============================] - 83s - loss: 5.4910 - acc: 0.0371 - val_loss: 4.9326 - val_acc: 0.0684
Epoch 2/20
2048/2048 [==============================] - 83s - loss: 4.5139 - acc: 0.1699 - val_loss: 4.5324 - val_acc: 0.1509
Epoch 3/20
2048/2048 [==============================] - 50s - loss: 4.1155 - acc: 0.2769 - val_loss: 4.2831 - val_acc: 0.2041
Epoch 4/20
2048/2048 [==============================] - 42s - loss: 3.8427 - acc: 0.3667 - val_loss: 4.1022 - val_acc: 0.2427
Epoch 5/20
2048/2048 [==============================] - 48s - loss: 3.6345 - acc: 0.4277 - val_loss: 3.9607 - val_acc: 0.2710
Epoch 6/20
2048/2048 [==============================] - 85s - loss: 3.4659 - acc: 0.4800 - val_loss: 3.8450 - val_acc: 0.2915
Epoch 7/20
2048/2048 [==============================] - 82s - loss: 3.3245 - acc: 0.5244 - val_loss: 3.7476 - val_acc: 0.3120
Epoch 8/20
2048/2048 [==============================] - 65s - loss: 3.

In [45]:
print(hist.history)

{'acc': [0.037109375, 0.169921875, 0.27685546875, 0.36669921875, 0.427734375, 0.47998046875, 0.5244140625, 0.5576171875, 0.58544921875, 0.60302734375, 0.62646484375, 0.6455078125, 0.662109375, 0.67578125, 0.6865234375, 0.697265625, 0.70703125, 0.716796875, 0.7236328125, 0.73388671875], 'loss': [5.490975484251976, 4.5138835534453392, 4.1154690161347389, 3.8427279517054558, 3.6344806142151356, 3.4659445509314537, 3.3244820944964886, 3.202712956815958, 3.0959329418838024, 3.0009526312351227, 2.9155075773596764, 2.8379308842122555, 2.7669586315751076, 2.7016091607511044, 2.6411044262349606, 2.584816750138998, 2.5322325453162193, 2.4829259403049946, 2.436539875343442, 2.392772126942873], 'val_acc': [0.068359375, 0.15087890625, 0.2041015625, 0.24267578125, 0.27099609375, 0.29150390625, 0.31201171875, 0.330078125, 0.341796875, 0.35107421875, 0.3671875, 0.373046875, 0.37939453125, 0.38671875, 0.39306640625, 0.39892578125, 0.408203125, 0.41162109375, 0.41650390625, 0.42236328125], 'val_loss': [